<a href="https://colab.research.google.com/github/Jagoda222/LoLa---group-8/blob/main/dataset_triplets_1000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sampling 10,000 Triplets from SNLI Dataset

This notebook demonstrates how to preprocess the SNLI dataset and sample **10,000 triplets** (30,000 rows). Each triplet consists of:
- One **premise** shared by three rows.
- Three **hypotheses** corresponding to labels `0` (entailment), `1` (neutral), and `2` (contradiction).

**Steps:**
1. Load the SNLI dataset
2. Assign unique **triplet numbers** to valid triplets (groups of three rows with the same premise).
3. Divide dataset into **10 equal-sized blocks** and randomly sample **1000 triplets** per block.
4. Combine sampled triplets and save the final dataset as `sampled_snli_triplets_10000.csv`.

This ensures uniform distribution and a clean triplet structure for further analysis.

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset

### Loading the dataset

In [7]:
dataset = load_dataset("snli")
train_data = dataset['train'].to_pandas()

# Rename columns for clarity
train_data = train_data.rename(columns={
    'sentence1': 'premise',
    'sentence2': 'hypothesis',
    'gold_label': 'label'
})

train_data = train_data[train_data['label'].notnull()]
print(train_data.head())
print(train_data.describe())

                                             premise  \
0  A person on a horse jumps over a broken down a...   
1  A person on a horse jumps over a broken down a...   
2  A person on a horse jumps over a broken down a...   
3              Children smiling and waving at camera   
4              Children smiling and waving at camera   

                                          hypothesis  label  
0  A person is training his horse for a competition.      1  
1      A person is at a diner, ordering an omelette.      2  
2                  A person is outdoors, on a horse.      0  
3                  They are smiling at their parents      1  
4                         There are children present      0  
               label
count  550152.000000
mean        0.996730
std         0.819796
min        -1.000000
25%         0.000000
50%         1.000000
75%         2.000000
max         2.000000


### Checking the label distribution

In [8]:
unique_labels = train_data['label'].unique()
print("Unique label types:", unique_labels)

label_counts = train_data['label'].value_counts()
print("\nLabel counts:")
print(label_counts)


Unique label types: [ 1  2  0 -1]

Label counts:
label
 0    183416
 2    183187
 1    182764
-1       785
Name: count, dtype: int64


### Assigning number to each triplet

In [9]:
triplet_nr = 1
triplet_numbers = []

# Iterate through the dataset in groups of three rows
for i in range(0, len(train_data), 3):
    group = train_data.iloc[i:i+3]

    # Check if all three rows share the same premise
    if len(group['premise'].unique()) == 1 and len(group) == 3:
        triplet_numbers.extend([triplet_nr] * 3)
        triplet_nr += 1
    else:
        triplet_numbers.extend([None] * len(group))

# Assign triplet numbers to the train_data
train_data['triplet_nr'] = triplet_numbers

# Drop rows without valid triplet assignment (optional, if you only want valid triplets)
train_data = train_data.dropna(subset=['triplet_nr'])

# Convert 'triplet_nr' to integer
train_data['triplet_nr'] = train_data['triplet_nr'].astype(int)

# Print the first few rows
print(train_data.head(9))


                                             premise  \
0  A person on a horse jumps over a broken down a...   
1  A person on a horse jumps over a broken down a...   
2  A person on a horse jumps over a broken down a...   
3              Children smiling and waving at camera   
4              Children smiling and waving at camera   
5              Children smiling and waving at camera   
6  A boy is jumping on skateboard in the middle o...   
7  A boy is jumping on skateboard in the middle o...   
8  A boy is jumping on skateboard in the middle o...   

                                          hypothesis  label  triplet_nr  
0  A person is training his horse for a competition.      1           1  
1      A person is at a diner, ordering an omelette.      2           1  
2                  A person is outdoors, on a horse.      0           1  
3                  They are smiling at their parents      1           2  
4                         There are children present      0          

<ipython-input-9-3942600669fd>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['triplet_nr'] = train_data['triplet_nr'].astype(int)


In [10]:
# Step 1: Identify triplet numbers containing -1 labels
triplets_with_neg1 = train_data[ train_data['label'] == -1]['triplet_nr'].unique()

# Step 2: Filter out all rows belonging to those triplets
train_data = train_data[~train_data['triplet_nr'].isin(triplets_with_neg1)]

# Step 3: Optional - Check the resulting cleaned data
print("Cleaned data without any triplets containing -1 labels:")
print( train_data.head(9))


Cleaned data without any triplets containing -1 labels:
                                             premise  \
0  A person on a horse jumps over a broken down a...   
1  A person on a horse jumps over a broken down a...   
2  A person on a horse jumps over a broken down a...   
3              Children smiling and waving at camera   
4              Children smiling and waving at camera   
5              Children smiling and waving at camera   
6  A boy is jumping on skateboard in the middle o...   
7  A boy is jumping on skateboard in the middle o...   
8  A boy is jumping on skateboard in the middle o...   

                                          hypothesis  label  triplet_nr  
0  A person is training his horse for a competition.      1           1  
1      A person is at a diner, ordering an omelette.      2           1  
2                  A person is outdoors, on a horse.      0           1  
3                  They are smiling at their parents      1           2  
4            

### Checking if the numbers of triplets were assigned correctly to the ones with label -1

In [12]:
print("Unique labels in train_data:", train_data['label'].unique())

triplets_with_neg1 = train_data[train_data['label'] == -1]['triplet_nr'].unique()

print(f"Total number of triplets containing label -1: {len(triplets_with_neg1)}")
triplets_with_neg1_data = train_data[train_data['triplet_nr'].isin(triplets_with_neg1)]

print("First 3 triplets containing at least one row with label -1:")
for triplet_nr in triplets_with_neg1[:3]:
    print(f"\nTriplet Number: {triplet_nr}")
    print(triplets_with_neg1_data[triplets_with_neg1_data['triplet_nr'] == triplet_nr])


Unique labels in train_data: [1 2 0]
Total number of triplets containing label -1: 0
First 3 triplets containing at least one row with label -1:


### Chosing 100 random triplets containing -1 label


### Dividing dataset (without triplets with -1 label) in 10 blocks. Choosing 990 random triplets from each block. Combining triplets with and without -1 label.

In [14]:
import numpy as np
import pandas as pd

num_blocks = 10
triplets_per_block = 1000

unique_triplets = train_data['triplet_nr'].unique()
triplets_without_neg1 = np.setdiff1d(unique_triplets, triplets_with_neg1)

# Calculate the size of each block
num_triplets = len(unique_triplets)
block_size = num_triplets // num_blocks

sampled_triplets = []

for i in range(num_blocks):
    start_idx = i * block_size
    end_idx = (i + 1) * block_size if i < num_blocks - 1 else num_triplets

    block_triplets = unique_triplets[start_idx:end_idx]

    block_triplets_without_neg1 = np.intersect1d(block_triplets, triplets_without_neg1)

    # Sample triplets from the current block
    sampled_without_neg1 = train_data[train_data['triplet_nr'].isin(block_triplets_without_neg1)]
    sampled_without_neg1 = sampled_without_neg1['triplet_nr'].drop_duplicates().sample(
        n=min(triplets_per_block, len(block_triplets_without_neg1)), random_state=42
    )
    sampled_without_neg1 = train_data[train_data['triplet_nr'].isin(sampled_without_neg1)]
    sampled_triplets.append(sampled_without_neg1)

# Combine all sampled triplets
sampled_triplets_combined = pd.concat(sampled_triplets)

# Check the number of unique triplets and label distribution
print(f"Total number of unique triplets chosen: {sampled_triplets_combined['triplet_nr'].nunique()}")
print("\nLabel distribution in the sampled data:")
print(sampled_triplets_combined['label'].value_counts())


Total number of unique triplets chosen: 10000

Label distribution in the sampled data:
label
0    10071
1     9982
2     9947
Name: count, dtype: int64


In [15]:
sampled_triplets_combined.to_csv('sampled_snli_10000.csv', index=False)
